In [28]:
import os
import requests
import json
##Create service accounts and tokens
#nucleus_uri = os.environ['DOMINO_API_HOST']
api_proxy = os.environ['DOMINO_API_PROXY']
svc_accounts_endpoint= f'{api_proxy}/v4/serviceAccounts'


## Why Service Account?

Service Accounts are industry standard ways of creating identities which only interact with the platform programmatically. They are identities used to execute automation workflows.


## What functions does this notebook expose?

1. List all service accounts - `get_all_service_accounts`
2. Create a new service account - `create_sa(sa_name)`
3. Get the Keycloak Id associated with each SA - `get_idp_id_by_sa()`
4. Get the Domino (Mongo) Id associated with each SA - `get_id_by_sa()`
5. Create a named token for a service account. A service account can have many named tokens -`create_token(idpid,token_name)`
6. Invalidate token for a service account. A token that is invalidated cannot be reactivated - `invalidate_token(token_name):`
7. Delete token for a service account. A token with the same name can be recreated after its deleted - `delete_token(token_name)`
8. Add a service account as a project collaborator (collab_id==sa_mongo_id) - `add_sa_as_project_collaborator(project_id,collab_id,project_role)`
9. Get git credential id's associated with all users (id==mongo_id)- `get_git_credentials(id)`
10. Get repo_id associated with a git backed project - `get_project_repo_id(project_id)`
11. Get git credentials associated with a give project - `get_git_credentials`
12. Add a git credential for a collaborating project for a given sa `add_git_cred_for_project_for_sa`
13. Delete git credentials for a collaborating project for a given sa `delete_git_credentials_for_sa`

(11,12,13) in the real world would be run by a SA token after the SA is added as a collaborator for a give project




In [29]:
## First step is fetch all service account
def  get_all_service_accounts():
    print(svc_accounts_endpoint)
    resp = requests.get(svc_accounts_endpoint)
    print(resp)
    users=[]
    if resp.status_code==200:
        for u in resp.json():
            users.append(u['username'])
    return users

In [30]:
print(get_all_service_accounts())

http://localhost:8899/v4/serviceAccounts
<Response [200]>
['svc-user-ds-u1', 'svc-admin-u2']


In [31]:
## Create Service Accounts if they don't exist

In [32]:
def create_sa(sa_name:str):
    data = {'username':sa_name,'email':f'{sa_name}@xyz.com'}
    resp = requests.post(svc_accounts_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(f'Response for creating svc account {sa_name} is {resp.status_code}')
    print(resp.json())
    

In [33]:
svc_accounts=['svc-user-ds-u1','svc-admin-u2']
for sa_name in svc_accounts:
    ## 200 
    create_sa(sa_name)

Response for creating svc account svc-user-ds-u1 is 500
{'message': 'A user with the username svc-user-ds-u1 already exists.', 'success': False}
Response for creating svc account svc-admin-u2 is 500
{'message': 'A user with the username svc-admin-u2 already exists.', 'success': False}


In [34]:
#Now get Idp Id of all emails. This only returns active service accounts. If you have deactivated the SA
#you can no longer see it. The only place to get that is from Mongo or Keycloak to reactivate
#Activation of SA using names is not supported
def get_idp_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['idpId']
    return idp_by_user
def get_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['id']
    return idp_by_user

In [35]:
idp_by_user = get_idp_id_by_sa()

[{'username': 'svc-user-ds-u1', 'email': 'svc-user-ds-u1@xyz.com', 'id': '68b9af2aef0f2d1f1e95fdc4', 'idpId': '99d06216-d4bd-444f-9b03-1ed57c28e8ab'}, {'username': 'svc-admin-u2', 'email': 'svc-admin-u2@xyz.com', 'id': '68b9af2b63f3790aeb9ba4b8', 'idpId': '2679d9fa-2efc-4787-a9a5-8800a885de13'}]


In [36]:
# You cannot overwrite an existing or even an invalidated token. You have to delete it first
def create_token(idpid,token_name):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    resp = requests.post(f"{token_endpoint}",json={"name":token_name},
                         headers={'Content-Type':'application/json'})
    print(resp.status_code)
    return resp.json()

In [45]:
results = []
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"    
    j = create_token(idpid,token_name)
    results.append(j)

with open("/tmp/output.json", "w") as file:
    json.dump(results, file, indent=2)

201
201


In [38]:
def deactivate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/deactivate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Deactivating user {user}")
def activate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/activate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Activating user {user}")

In [39]:
for user, idpid in idp_by_user.items():
    deactivate_sa(idpid)
for user, idpid in idp_by_user.items():
    activate_sa(idpid)

Status code 200 - Deactivating user svc-user-ds-u1
Status code 200 - Deactivating user svc-admin-u2
Status code 200 - Activating user svc-user-ds-u1
Status code 200 - Activating user svc-admin-u2


In [40]:
#List all tokens
def list_all_tokens(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    
    resp = requests.get(token_endpoint,headers={'Content-Type':'application/json'})
    
    if resp.status_code==200:
       out = resp.json()           
       pretty_json = json.dumps(out, indent=4)
       print(pretty_json)



In [41]:
for user, idpid in idp_by_user.items():
    list_all_tokens(idpid)

[
    {
        "name": "svc-user-ds-u1-token-app",
        "serviceAccountIdpId": "99d06216-d4bd-444f-9b03-1ed57c28e8ab",
        "isValid": true,
        "createdAt": "2025-09-04T19:00:10.018Z",
        "expiresAt": "2026-01-02T19:00:10.005Z"
    }
]
[
    {
        "name": "svc-admin-u2-token-app",
        "serviceAccountIdpId": "2679d9fa-2efc-4787-a9a5-8800a885de13",
        "isValid": true,
        "createdAt": "2025-09-04T19:00:10.426Z",
        "expiresAt": "2026-01-02T19:00:10.413Z"
    }
]


In [42]:
def invalidate_token(token_name):
    print(f"Invalidating token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.post(f"{token_endpoint}/invalidate",headers={'Content-Type':'application/json'})
    print(resp.status_code)
def delete_token(token_name):
    print(f"Deleting token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.delete(f"{token_endpoint}",headers={'Content-Type':'application/json'})
    print(resp.status_code)
    

In [44]:
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # Invalidated tokens cannot be used or be revalidated
    invalidate_token(token_name)

for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # You have to delete a token to recreate it
    delete_token(token_name)

Invalidating token svc-user-ds-u1-token-app?
200
Invalidating token svc-admin-u2-token-app?
200
Deleting token svc-user-ds-u1-token-app?
200
Deleting token svc-admin-u2-token-app?
200


In [ ]:

def add_sa_as_project_collaborator(project_id,collab_id,project_role):
    project_endpoint= f'{api_proxy}/v4/projects/{project_id}/collaborators'
    data = {"collaboratorId":collab_id,
            "projectRole":project_role}
    resp = requests.post(url=project_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(resp.status_code)
    print(resp.json())
    

In [ ]:
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
for u,id in id_by_user.items():
    add_sa_as_project_collaborator(project_id,id,"Contributor")


In [ ]:
## Now we need to add a git credential to a service account

In [60]:
def delete_git_credentials_for_sa(id,credential_id):
    
    headers = {'Content-Type':'application/json'}
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials/{credential_id}"    
    resp = requests.delete(url,headers=headers)   
    print(resp.status_code)
    return resp.json()

In [61]:
def add_git_credentials_to_sa(id,git_token_name,git_token):
    headers = {'Content-Type':'application/json'}    
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"
    p = {"name":git_token_name,"gitServiceProvider":"github","accessType":"token", "token":git_token,"type":"TokenGitCredentialDto"}
    resp = requests.post(url,json=p,headers=headers)   
    print(resp.status_code)
    return resp.json()


In [62]:
def get_git_credentials(id):    
    headers = {'Content-Type':'application/json'}    
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"    
    resp = requests.get(url,headers=headers)   
    print(resp.status_code)
    print(resp.text)
    return resp.json()


In [63]:
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
svc_token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI2ekZfWlgzRlJiV0dnT215VHJKd0xmXzFVNUxUeENVT0t4bTNvSDhZNW5ZIn0.eyJleHAiOjE3NjczODIxMTMsImlhdCI6MTc1NzAxNDExMywianRpIjoiYjhkYWM4MzEtZTcwYS00NmZjLTlhNWYtYjUxNzU0M2IwNjkyIiwiaXNzIjoiaHR0cHM6Ly9tYXJjZG83NzM2NC5jcy5kb21pbm8udGVjaC9hdXRoL3JlYWxtcy9Eb21pbm9SZWFsbSIsImF1ZCI6WyJkb21pbm8tcGxhdGZvcm0iLCJhY2NvdW50Il0sInR5cCI6IkJlYXJlciIsImF6cCI6ImRvbWluby1zZXJ2aWNlLWFjY291bnRzIiwic2lkIjoiMzhlMjU2NjYtNGY3Yy00ZmY1LWJiNjAtNmIxYTJlNjJiMGE4IiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLWRvbWlub3JlYWxtIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiZG9taW5vLXNlcnZpY2UtYWNjb3VudHMiOnsicm9sZXMiOlsiZG9taW5vLXNlcnZpY2UtYWNjb3VudCJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZG9taW5vLWp3dC1jbGFpbXMgZW1haWwgb2ZmbGluZV9hY2Nlc3MgcHJvZmlsZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiaWRwX2lkIjoiOTlkMDYyMTYtZDRiZC00NDRmLTliMDMtMWVkNTdjMjhlOGFiIiwibmFtZSI6IlNBIHN2Yy11c2VyLWRzLXUxIiwicHJlZmVycmVkX3VzZXJuYW1lIjoic3ZjLXVzZXItZHMtdTEiLCJnaXZlbl9uYW1lIjoiU0EiLCJmYW1pbHlfbmFtZSI6InN2Yy11c2VyLWRzLXUxIiwiZW1haWwiOiJzdmMtdXNlci1kcy11MUB4eXouY29tIiwidXNlcl9ncm91cHMiOlsiL3JvbGVzL1ByYWN0aXRpb25lciJdfQ.my59LrgU1AOcSTVMwoJt4_Gg7crsT578sVDPI3IrHpAAYec4oQi_b2qbABPcYTz8SJ-SHvFk5Xy45tm-pCJgdwOeiiSY5NLkmqv1VUM7af6ponkRqOl5cIfJLyNdmCc_KeRp46_kcBtRhHwGUUfq8lSc7C4buws9aCQxc9ehwW59h6tHZv2dwuxby79d7WdobvbwicOvHB4it3bxTq_bM7JnRAxLEf09Ktr5hDMW8xBQ3bcgO7Vd2VnBJCtg70Y6QaCQwVb08O3gRrCqovOrF6vbUzNsjBxIXaF2h4ikufquUhs0pc0mPw2UYDCctMdwQut5b9VaOzlhgFGf6yxq-A"

token_name = "my-token"
u = "svc-user-ds-u1"
id = "68b9af2aef0f2d1f1e95fdc4"

print(f"{u} - {id}")
creds = get_git_credentials(id)

for c in creds:    
    credential_id = c['id']
    delete_git_credentials_for_sa(id,credential_id)
    print(add_git_credentials_to_sa(id,token_name,os.environ['SVC_GITHUB_TOKEN']))


[{'username': 'svc-user-ds-u1', 'email': 'svc-user-ds-u1@xyz.com', 'id': '68b9af2aef0f2d1f1e95fdc4', 'idpId': '99d06216-d4bd-444f-9b03-1ed57c28e8ab'}, {'username': 'svc-admin-u2', 'email': 'svc-admin-u2@xyz.com', 'id': '68b9af2b63f3790aeb9ba4b8', 'idpId': '2679d9fa-2efc-4787-a9a5-8800a885de13'}]
svc-user-ds-u1 - 68b9af2aef0f2d1f1e95fdc4
200
[{"id":"68b9c8b7ef0f2d1f1e95fdcb","name":"fake-name","gitServiceProvider":"github","domain":"github.com","fingerprint":"7b:3c:af:44:d3:37:ca:e1:74:54:0c:4d:26:e2:2b:27","protocol":"https"}]
200
200
{'id': '68b9ea4463f3790aeb9ba562', 'name': 'my-token', 'gitServiceProvider': 'github', 'domain': 'github.com', 'fingerprint': 'ba:29:6e:74:6f:9f:fd:b1:e0:f6:18:dd:95:b1:94:fc', 'protocol': 'https'}


In [69]:
def get_project_repo_id(project_id,token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    
    url = f"{api_proxy}/v4/projects/{project_id}"    
    obj = requests.get(url,headers=headers).json()
    print(obj)
    git_repository_id = obj['mainRepository']['id']
    return git_repository_id

##Run this using the service account token. This is for illustration only
def add_git_cred_for_project_for_sa(project_id,repo_id,cred_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_host}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.put(url,json={"credentialId": cred_id},headers=headers)
    print(out.status_code)
    print(out.text)

def get_git_cred_for_project_for_sa(project_id,repo_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.get(url,headers=headers)
    print(out.status_code)
    print(out.text)

def delete_git_cred_for_project_for_sa(project_id,repo_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.delete(url,headers=headers)
    print(out.status_code)
    print(out.text)



In [70]:

project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
repo_id = get_project_repo_id(project_id,svc_token)
print(f"Repo id {repo_id}")

[{'username': 'svc-user-ds-u1', 'email': 'svc-user-ds-u1@xyz.com', 'id': '68b9af2aef0f2d1f1e95fdc4', 'idpId': '99d06216-d4bd-444f-9b03-1ed57c28e8ab'}, {'username': 'svc-admin-u2', 'email': 'svc-admin-u2@xyz.com', 'id': '68b9af2b63f3790aeb9ba4b8', 'idpId': '2679d9fa-2efc-4787-a9a5-8800a885de13'}]
{'message': 'The JWT must contain a subject or an id must be generated via the identifierGenerator', 'success': False}


KeyError: 'mainRepository'

In [ ]:
##This should be run using each service account tokens 
for u,id in id_by_user.items():
    creds = get_git_credentials(id)
    credential_id = creds[0]['id']
    print("Get Git Cred mapping")
    print(get_git_cred_for_project_for_sa(project_id,repo_id))

    print("Delete Git Cred mapping")
    delete_git_cred_for_project_for_sa(project_id,repo_id)

    print("Add Git Cred mapping")
    add_git_cred_for_project_for_sa(project_id,repo_id,credential_id)

In [ ]:
## And now you can use the service account token to run jobs for a git backed project

In [46]:
import base64
import json

def decode_jwt(token: str):
    header_b64, payload_b64, signature_b64 = token.split('.')

    # Base64URL decoding requires padding correction
    def b64url_decode(b64_str):
        padding = '=' * (-len(b64_str) % 4)
        return base64.urlsafe_b64decode(b64_str + padding)

    header = json.loads(b64url_decode(header_b64))
    payload = json.loads(b64url_decode(payload_b64))
    
    return {
        "header": header,
        "payload": payload,
        "signature": signature_b64  # raw, usually verified separately
    }

# Example usage
decoded = decode_jwt(svc_token)
print(json.dumps(decoded, indent=2))

{
  "header": {
    "alg": "RS256",
    "typ": "JWT",
    "kid": "6zF_ZX3FRbWGgOmyTrJwLf_1U5LTxCUOKxm3oH8Y5nY"
  },
  "payload": {
    "exp": 1767382113,
    "iat": 1757014113,
    "jti": "b8dac831-e70a-46fc-9a5f-b517543b0692",
    "iss": "https://marcdo77364.cs.domino.tech/auth/realms/DominoRealm",
    "aud": [
      "domino-platform",
      "account"
    ],
    "typ": "Bearer",
    "azp": "domino-service-accounts",
    "sid": "38e25666-4f7c-4ff5-bb60-6b1a2e62b0a8",
    "realm_access": {
      "roles": [
        "offline_access",
        "uma_authorization",
        "default-roles-dominorealm"
      ]
    },
    "resource_access": {
      "domino-service-accounts": {
        "roles": [
          "domino-service-account"
        ]
      },
      "account": {
        "roles": [
          "manage-account",
          "manage-account-links",
          "view-profile"
        ]
      }
    },
    "scope": "openid domino-jwt-claims email offline_access profile",
    "email_verified": false,
 